## 一、异常点清洗
1. 删除坐标漂移点：通常GPS坐标漂移表现为经纬度值异常大或异常小
2. 删除数据缺失点：检查是否有缺失值（NaN或空值）

In [4]:
# 相关包导入
import pandas as pd
import os
from pathlib import Path
import logging

In [2]:
def clean_gps_data(df, file_name):
    # 定义合理的经纬度范围
    # 以下以上海市为例
    LAT_MIN, LAT_MAX = 30.40, 31.53  # 纬度范围
    LON_MIN, LON_MAX = 120.52, 122.12  # 经度范围
    
    # 记录原始数据量
    original_count = len(df)
    
    # 删除缺失值并记录
    missing_mask = df.isnull().any(axis=1)
    df = df.dropna()
    missing_count = missing_mask.sum()
    if missing_count > 0:
        logging.info(f"文件 {file_name} 删除 {missing_count} 个缺失值")
    
    # 删除坐标漂移点并记录
    invalid_mask = ~((df['latitude'].between(LAT_MIN, LAT_MAX)) & 
                    (df['longitude'].between(LON_MIN, LON_MAX)))
    df = df[~invalid_mask]
    invalid_count = invalid_mask.sum()
    if invalid_count > 0:
        logging.info(f"文件 {file_name} 删除 {invalid_count} 个坐标漂移点")
    
    # 记录清洗结果
    cleaned_count = len(df)
    if original_count > cleaned_count:
        logging.info(f"文件 {file_name} 清洗比例: {(original_count - cleaned_count)/original_count:.2%}")
    
    return df

In [3]:
def setup_logger():
    # 配置日志
    logging.basicConfig(
        filename='data_cleaning.log',
        level=logging.INFO,
        format='%(asctime)s - %(message)s'
    )

In [4]:
def process_taxi_data(folder_path):
    setup_logger()
    # 初始化统计变量
    total_taxis = 0
    total_points_before = 0
    total_points_after = 0
    
    # 遍历文件夹中的所有CSV文件
    for file in Path(folder_path).glob('*.csv'):
        total_taxis += 1
        
        # 读取数据
        df = pd.read_csv(file)
        total_points_before += len(df)
        
        # 清洗数据
        cleaned_df = clean_gps_data(df, file.name)
        total_points_after += len(cleaned_df)
        
        # # 保存清洗后的数据（可选）
        # cleaned_df.to_csv(f'cleaned_{file.name}', index=False)
    
    # 打印统计结果
    print(f"出租车总数: {total_taxis}")
    print(f"清洗前轨迹点总数: {total_points_before}")
    print(f"清洗后轨迹点总数: {total_points_after}")
    print(f"清洗掉的数据比例: {(total_points_before - total_points_after)/total_points_before:.2%}")

# 用于测试清洗结果
# process_taxi_data('./data/vehicles')

**数据清洗结果**:

出租车总数: 6831

清洗前轨迹点总数: 27093007

清洗后轨迹点总数: 25509125

清洗掉的数据比例: 5.85%

## 二、出租车出行记录提取
根据我们的数据集的结构，一个GPS采样点的信息可以用五元组$<id, x, y, t, s>$表示，其中：

* $id$: 出租车车牌的唯一映射;
* $x$: 出租车坐标的经度;
* $y$: 出租车坐标的纬度;
* $t$: 该GPS采样点的时间戳(相对于当日0点的时间，单位：秒);
* $s$: 出租车的载客状态(0表示载客，1表示空车，2表示空车但又任务);

我们暂且提取其中的某一辆车的一组连续的“载客状态”下的轨迹作为一次有效出行，用于研究人群的出行行为；

In [5]:
def extract_trips(df, vehicle_id):
    # 按时间戳排序
    df = df.sort_values('time')

    # 初始化
    trips = []
    current_trip = []
    
    # 提取车辆ID的数字部分
    vehicle_num = int(''.join(filter(str.isdigit, vehicle_id)))
    
    # 遍历每个GPS点
    for _, row in df.iterrows():
        # 如果是载客状态
        if row['status'] == 0:
            # 创建五元组 (id, x, y, t, s)
            point = (
                vehicle_num,  # 整数ID
                row['longitude'],  # 保持浮点数
                row['latitude'],  # 保持浮点数
                int(row['time']),  # 转换为整数
                int(row['status'])  # 转换为整数
            )
            current_trip.append(point)
        else:
            # 如果当前有载客轨迹，保存并重置
            if len(current_trip) > 0:
                trips.append(current_trip)
                current_trip = []
    
    # 处理最后一个可能的载客轨迹
    if len(current_trip) > 0:
        trips.append(current_trip)
    
    return trips

In [6]:
def save_trips(trips, output_folder, vehicle_id):
    # 为每辆车创建输出文件夹
    vehicle_num = int(''.join(filter(str.isdigit, vehicle_id)))
    vehicle_folder = os.path.join(output_folder, f"vehicle_{str(vehicle_num)}")
    os.makedirs(vehicle_folder, exist_ok=True)
    
    # 保存每个载客轨迹
    for i, trip in enumerate(trips):
        trip_df = pd.DataFrame(trip, columns=['id', 'x', 'y', 't', 's'])
        # 确保数据类型
        trip_df = trip_df.astype({
            'id': 'int32',
            't': 'int32',
            's': 'int8'
        })
        trip_df.to_csv(os.path.join(vehicle_folder, f'trip_{i+1}.csv'), index=False)

In [7]:
def process_vehicle_data(folder_path, output_folder):
    # 遍历所有车辆文件
    for file in Path(folder_path).glob('*.csv'):
        # 读取数据
        df = pd.read_csv(file)
        vehicle_id = file.stem  # 从文件名获取车辆ID
        
        # 清洗数据
        cleaned_df = clean_gps_data(df, file.name)

        # 提取载客轨迹
        trips = extract_trips(cleaned_df, vehicle_id)
        
        # 保存载客轨迹
        save_trips(trips, output_folder, vehicle_id)

In [8]:
# 进行异常点筛除并提取载客轨迹
process_vehicle_data('./data/vehicles', './data/trips')

## 三、出行记录清洗
需要对提取出的出行记录中质量较差的轨迹进行筛除，标准如下：
1. GPS点过少($<5$);
2. 速度异常($>120km/h$);
3. 距离过短($<500m$);

In [12]:
import numpy as np

In [13]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # 计算两个GPS点之间的距离（单位：米）
    R = 6371000  # 地球半径，单位：米
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat/2) * np.sin(dlat/2) +
        np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) *
        np.sin(dlon/2) * np.sin(dlon/2))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

In [15]:
def clean_trip(trip):
    if not isinstance(trip[0], (list, tuple)):
        trip = [trip]
    
    # 规则1：GPS点过少
    if len(trip) < 5:
        return []
    
    # 计算每个点的速度
    speeds = []
    distances = []
    for i in range(1, len(trip)):
        # 确保每个点有5个值
        if len(trip[i-1]) < 5 or len(trip[i]) < 5:
            continue
            
        # 计算距离
        dist = calculate_distance(
            float(trip[i-1][2]), float(trip[i-1][1]),  # 前一点的纬度，经度
            float(trip[i][2]), float(trip[i][1])       # 当前点的纬度，经度
        )
        distances.append(dist)
        
        # 计算时间差（秒）
        time_diff = float(trip[i][3]) - float(trip[i-1][3])
        if time_diff == 0:
            speed = 0
        else:
            speed = dist / time_diff * 3.6  # 转换为km/h
        
        speeds.append(speed)
    
    # 规则2：速度异常
    if any(s > 120 for s in speeds):
        return []
        
    # 规则3：距离过短
    total_distance = sum(distances)
    if total_distance < 500:
        return []
        
    # 通过所有检查，保留该轨迹
    return trip

In [16]:
def process_trips_folder(input_folder, output_folder):
    total_trips = 0
    removed_trips = 0
    
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    # 遍历所有车辆文件夹
    for vehicle_dir in Path(input_folder).iterdir():
        if not vehicle_dir.is_dir():
            continue
            
        # 创建对应的输出文件夹
        cleaned_vehicle_dir = os.path.join(output_folder, vehicle_dir.name)
        os.makedirs(cleaned_vehicle_dir, exist_ok=True)
        
        # 遍历每个轨迹文件
        for trip_file in vehicle_dir.glob('*.csv'):
            total_trips += 1
            
            # 读取数据
            df = pd.read_csv(trip_file)
            
            # 清洗数据
            cleaned_trip = clean_trip(df.values.tolist())
            
            if len(cleaned_trip) == 0:
                removed_trips += 1
                continue
                
            # 保存清洗后的数据
            cleaned_df = pd.DataFrame(cleaned_trip, columns=['id', 'x', 'y', 't', 's'])
            output_path = os.path.join(cleaned_vehicle_dir, trip_file.name)
            cleaned_df.to_csv(output_path, index=False)
    
    # 计算统计信息
    removal_rate = removed_trips / total_trips if total_trips > 0 else 0
    
    print(f"总轨迹数: {total_trips}")
    print(f"删除轨迹数: {removed_trips}")
    print(f"删除比例: {removal_rate:.2%}")

In [17]:
input_folder = './data/trips'
output_folder = './data/cleaned_trips'
process_trips_folder(input_folder, output_folder)

总轨迹数: 115564
删除轨迹数: 23272
删除比例: 20.14%


In [ ]:
def reorganize_trips(input_folder, output_folder):
    trip_counter = 1  # 用于连续编号
    
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    # 遍历所有车辆文件夹
    for vehicle_dir in Path(input_folder).iterdir():
        if not vehicle_dir.is_dir():
            continue
            
        # 遍历每个轨迹文件
        for trip_file in vehicle_dir.glob('*.csv'):
            # 读取数据
            df = pd.read_csv(trip_file)
            
            # 保存到新的文件
            output_path = os.path.join(output_folder, f'trip_{trip_counter}.csv')
            df.to_csv(output_path, index=False)
            trip_counter += 1
    
input_folder = './data/cleaned_trips'
output_folder = './data/cleaned_trips_combined'
reorganize_trips(input_folder, output_folder)

## 四、源数据提取
暂且不关注出租车的行驶轨迹，仅关心人群的出行行为，将出行记录简化为1个5元组$<x_o, y_o, x_d, y_d, d>$表示，其中:
* $x_o, y_o$: 分别表示起点的经纬度;
* $x_d, y_d$: 分别表示终点的经纬度;
* $d$: 表示行程距离;

In [14]:
def simplify_trip(trip):
    if len(trip) < 2:
        return None
        
    # 获取起点和终点
    start_point = trip[0]
    end_point = trip[-1]
    
    # 计算总距离
    total_distance = 0
    for i in range(1, len(trip)):
        total_distance += calculate_distance(
            trip[i-1][2], trip[i-1][1],  # 前一点的纬度，经度
            trip[i][2], trip[i][1]       # 当前点的纬度，经度
        )
    
    return [
        start_point[1],  # x_o
        start_point[2],  # y_o
        end_point[1],    # x_d
        end_point[2],    # y_d
        total_distance   # d
    ]

In [15]:
def process_simplified_trips(input_folder, output_folder):
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    # 初始化统计
    total_trips = 0
    simplified_trips = []
    
    # 遍历所有轨迹文件
    for trip_file in Path(input_folder).glob('*.csv'):
        total_trips += 1
        
        # 读取数据
        df = pd.read_csv(trip_file)
        trip_data = df.values.tolist()
        
        # 简化出行记录
        simplified_trip = simplify_trip(trip_data)
        if simplified_trip:
            simplified_trips.append(simplified_trip)
    
    # 保存简化后的数据
    simplified_df = pd.DataFrame(
        simplified_trips,
        columns=['x_o', 'y_o', 'x_d', 'y_d', 'd']
    )
    output_path = os.path.join(output_folder, 'simplified_trips.csv')
    simplified_df.to_csv(output_path, index=False)
    
    print(f"成功简化了 {len(simplified_trips)} 个出行记录")
    print(f"简化后的数据已保存到 {output_path}")

In [16]:
input_folder = './data/cleaned_trips_combined'
output_folder = './data/simplified_trips'
process_simplified_trips(input_folder, output_folder)

成功简化了 92292 个出行记录
简化后的数据已保存到 ./data/simplified_trips/simplified_trips.csv
